In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/HIV_inhibitors_classification_and_generation/research'

In [2]:
os.chdir("../")
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/HIV_inhibitors_classification_and_generation'

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import numpy as np
from rdkit import Chem
import deepchem as dc

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torchdata.datapipes'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/HIV_inhibitors_classification_and_generation/env/lib/python3.10/site-packages/deepchem/mod

In [ ]:
hiv = pd.read_csv('artifacts/data_ingestion/HIV.csv')

In [ ]:
hiv

In [ ]:
def t_t_split_balance(df, split_size):
    # Separate positive and negative cases
    p_val = df[df.HIV_active == 1].to_numpy()
    n_val = df[df.HIV_active == 0].to_numpy()

    # Ensure class balance by selecting the smaller group as the target
    if len(p_val) >= len(n_val):
        big, small = p_val, n_val
    else:
        big, small = n_val, p_val

    # Stratified test split
    small_train, small_test = train_test_split(small, test_size=split_size, random_state=42)
    big_train, big_test = train_test_split(big, test_size=(split_size * len(small) / len(big)), random_state=42)

    test = np.concatenate([small_test, big_test])
    
    # Ensure the train set remains balanced by oversampling the smaller class
    train = np.concatenate([big_train, random.choices(small_train, k=len(big_train) - len(small_train))])

    # Convert back to DataFrame
    train_df = pd.DataFrame(train, columns=df.columns)
    test_df = pd.DataFrame(test, columns=df.columns)

    return train_df.sample(frac=1, random_state=42), test_df.sample(frac=1, random_state=42)

In [ ]:
train_df, test_df = t_t_split_balance(hiv, 0.2)

In [ ]:
train_df.head()

In [ ]:
p_list = p_df.values.tolist()

In [ ]:
test = random.choices(p_list, k=int(0.2*len(p_list)))
test

In [ ]:
test = pd.DataFrame(test)

In [ ]:
for row in hiv.head().iterrows():
    if hiv.eq(row[1]).all(axis=1).any():
        print(True)

In [ ]:
if list(test_df.columns) == ['smils', 'activity', 'HIV_active']:
    print(True)

In [ ]:
test_df['HIV_active'][0] = None

In [ ]:
bool(test_df['HIV_active'].isnull().sum())

In [6]:
train_pth = 'artifacts/data_transformation/train.csv'

In [10]:
file = train_pth.split('/')[-1].split('.')[0]
file

'train'

In [12]:
train = pd.read_csv('artifacts/data_transformation/train.csv')

In [13]:
train.smiles[0]

'CCCc1cc(=O)oc2c3c(cc(OC(C)CC)c12)OC(C)C(C)C3O'

In [14]:
mol_obj = Chem.MolFromSmiles(train.smiles[1])

In [ ]:
for atom in mol_obj.GetAtoms():
    print(atom.GetSymbol(), atom.GetChiralTag())

In [ ]:
bonds = mol_obj.GetBonds()

for bond in bonds:
    print(bond.GetBeginAtom().GetAtomicNum())
    print(bond.GetIsConjugated())
    print(bond.GetEndAtom().GetAtomicNum())
    print('___')

In [22]:
featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
f = featurizer._featurize(mol_obj)
# data = f.to_pyg_graph()
f.node_features

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0

In [23]:
datasets = {name: pd.read_csv(csv) for name, csv in 
                zip(["train", "test"], ['artifacts/data_transformation/train.csv', 'artifacts/data_transformation/test.csv'])}

In [24]:
datasets

{'train':                                                   smiles activity  HIV_active
 0          CCCc1cc(=O)oc2c3c(cc(OC(C)CC)c12)OC(C)C(C)C3O       CM           1
 1                Cn1c2ccccc2n2c(=O)c(-c3ccc(Cl)cc3)nnc12       CI           0
 2                  COc1ccc(NC(=O)CC(=O)Nc2ccc(OC)cc2)cc1       CI           0
 3      CC(C)CCCC(C)C1CCC2C3CCC4CC(CCC=C(c5cc(Br)c(O)c...       CA           1
 4                               COC=C1C(=O)OC(C)(C)OC1=O       CM           1
 ...                                                  ...      ...         ...
 77631                Cn1c2ccccc2c2ccc3c(c21)C(=O)C=CC3=O       CI           0
 77632                   Clc1cccc(Cl)c1C1SCc2nc3ncccc3n21       CM           1
 77633  CCC(=O)CCCC=C(c1cc(Cl)c(OC)c(C(=O)OC)c1)c1cc(C...       CA           1
 77634          CN1C(=O)C(=NN(c2ccccc2)c2ccccc2)c2ccccc21       CI           0
 77635  CCOC(=O)C(Cc1c(C=O)[nH]c2ccccc12)(NC(=O)c1cccc...       CI           0
 
 [77636 rows x 3 columns],
 'test':      